# Data Preprocessing and Combination with the Auxiliary Data

This file is used to train the model and generate the prediction result.

In [1]:
from src import preprocess
import pandas as pd

In [2]:
# Load the train dataset
file_path = './data/raw/train.csv'
df = pd.read_csv(file_path)

# Load the test dataset
file_path = './data/raw/test.csv'
df_test = pd.read_csv(file_path)


## Data Preprocessing

According to the exploratory data analysis, we will do the following data preprocessing.

#### Remove features that have only one value

In [3]:
df = preprocess.remove_no_use_features(df)
df_test = preprocess.remove_no_use_features(df_test)

# Show some basic statistics about the dataset
print('Shape of the dataset: ', df.shape)
df.head()


Shape of the dataset:  (60000, 14)


,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent
0,2021-09,jurong east,257,Jurong East Street 24,3 room,new generation,67.0,1983,1.344518,103.738630,yuhua east,jurong east,west region,1600
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,103.938717,bedok north,bedok,east region,2250
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,103.845643,toa payoh central,toa payoh,central region,1900
3,2021-08,pasir ris,250,Pasir Ris Street 21,executive,apartment,149.0,1993,1.370239,103.962894,pasir ris drive,pasir ris,east region,2850
4,2022-11,kallang/whampoa,34,Whampoa West,3-room,improved,68.0,1972,1.320502,103.863341,bendemeer,kallang,central region,2100


#### Convert categorical features to numerical features

In [4]:
df, df_test = preprocess.encode_features(df, df_test)

# Show some basic statistics about the dataset
print('Shape of the dataset: ', df.shape)
df.head()


Shape of the dataset:  (60000, 14)


,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent
0,0.67,12,553,239,3,10,67.0,1983,1.344518,103.738630,149,12,4,1600
1,1.33,1,88,577,4,10,92.0,1978,1.330186,103.938717,8,1,1,2250
2,1.75,23,186,828,3,5,67.0,1971,1.332242,103.845643,128,26,0,1900
3,0.58,16,537,328,1,3,149.0,1993,1.370239,103.962894,91,18,1,2850
4,1.83,14,952,475,3,5,68.0,1972,1.320502,103.863341,12,14,0,2100


#### Remove outliers

Using IQR based on Mahalanobis distance of each data point to remove outliers.

In [5]:
df_mahalanobis = preprocess.IQR(df)

print('Shape of the dataset after removing outliers using IQR: ', df_mahalanobis.shape)
df_mahalanobis.head()


Shape of the dataset after removing outliers using IQR:  (57509, 14)


,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent
0,0.67,12,553,239,3,10,67.0,1983,1.344518,103.738630,149,12,4,1600
1,1.33,1,88,577,4,10,92.0,1978,1.330186,103.938717,8,1,1,2250
2,1.75,23,186,828,3,5,67.0,1971,1.332242,103.845643,128,26,0,1900
3,0.58,16,537,328,1,3,149.0,1993,1.370239,103.962894,91,18,1,2850
4,1.83,14,952,475,3,5,68.0,1972,1.320502,103.863341,12,14,0,2100


Using Isolation Forest to remove outliers.

In [6]:
df_iso_forest = preprocess.IoslationForest(df)

# Show some basic statistics about the dataset after removing outliers
print('Shape of the dataset after removing outliers using Isolation Forest: ', df_iso_forest.shape)
df_iso_forest.head()


Shape of the dataset after removing outliers using Isolation Forest:  (59400, 14)


,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent
0,0.67,12,553,239,3,10,67.0,1983,1.344518,103.738630,149,12,4,1600
1,1.33,1,88,577,4,10,92.0,1978,1.330186,103.938717,8,1,1,2250
2,1.75,23,186,828,3,5,67.0,1971,1.332242,103.845643,128,26,0,1900
3,0.58,16,537,328,1,3,149.0,1993,1.370239,103.962894,91,18,1,2850
4,1.83,14,952,475,3,5,68.0,1972,1.320502,103.863341,12,14,0,2100


In [7]:
df = preprocess.remove_outliers(df)

#### Combine the auxiliary data

Location relevant data: shopping malls, mrt stations, primary school.

In [8]:
df = preprocess.AddAuxiliaryFeatures(df, radius=1, shopping_malls_path='./data/raw/auxiliary-data/sg-shopping-malls.csv',
                                        mrt_planned_path='./data/raw/auxiliary-data/sg-mrt-planned-stations.csv',
                                        mrt_existing_path='./data/raw/auxiliary-data/sg-mrt-existing-stations.csv',
                                        primary_schools_path='./data/raw/auxiliary-data/sg-primary-schools.csv')
df_test = preprocess.AddAuxiliaryFeatures(df_test, radius=1, shopping_malls_path='./data/raw/auxiliary-data/sg-shopping-malls.csv',
                                        mrt_planned_path='./data/raw/auxiliary-data/sg-mrt-planned-stations.csv',
                                        mrt_existing_path='./data/raw/auxiliary-data/sg-mrt-existing-stations.csv',
                                        primary_schools_path='./data/raw/auxiliary-data/sg-primary-schools.csv')

# Show some basic statistics about the dataset
print('Shape of the dataset: ', df.shape)
df.head()

Shape of the dataset:  (59400, 18)


,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent,Shopping Malls,Planned MRT Stations,Existing MRT Stations,Primary Schools
0,0.67,12,553,239,3,10,67.0,1983,1.344518,103.738630,149,12,4,1600,-1.202674,1.324998,0.300873,2.665154
1,1.33,1,88,577,4,10,92.0,1978,1.330186,103.938717,8,1,1,2250,-1.114338,0.095656,0.101009,4.392284
2,1.75,23,186,828,3,5,67.0,1971,1.332242,103.845643,128,26,0,1900,2.531703,-1.183876,3.781397,1.574240
3,0.58,16,537,328,1,3,149.0,1993,1.370239,103.962894,91,18,1,2850,0.597641,0.483794,-1.546040,2.435031
4,1.83,14,952,475,3,5,68.0,1972,1.320502,103.863341,12,14,0,2100,-1.073354,-2.818105,2.812144,1.728277


#### Dimensionality reduction

Combine the features that have similar meaning, for example:

- `town`, `block`, `street_name` are all related to the location of the flat,
- `floor_area_sqm`, `flat_type`, `flat_model` are all related to the size of the flat.
- `subzone`, `planning_area`, `region` are all related to more general location of the flat.

In [9]:
df, df_test = preprocess.dimensionality_reduction(df, df_test)

# Show some basic statistics about the dataset
print('Shape of the dataset: ', df.shape)
df.head()

Shape of the dataset:  (59400, 14)


,rent_approval_date,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,monthly_rent,Shopping Malls,Planned MRT Stations,Existing MRT Stations,Primary Schools,location_pca,area_features_pca
0,0.67,3,10,67.0,1983,1.344518,103.738630,1600,-1.202674,1.324998,0.300873,2.665154,1.018614,-0.585337
1,1.33,4,10,92.0,1978,1.330186,103.938717,2250,-1.114338,0.095656,0.101009,4.392284,1.340089,1.979528
2,1.75,3,5,67.0,1971,1.332242,103.845643,1900,2.531703,-1.183876,3.781397,1.574240,-1.128307,-1.838915
3,0.58,1,3,149.0,1993,1.370239,103.962894,2850,0.597641,0.483794,-1.546040,2.435031,0.482903,-0.554034
4,1.83,3,5,68.0,1972,1.320502,103.863341,2100,-1.073354,-2.818105,2.812144,1.728277,0.201158,0.801163


#### Feature normalization

In [10]:
df, df_test = preprocess.stupidNormalize(df, df_test)

# Show some basic statistics about the dataset
print('Shape of the dataset: ', df.shape)
df.head()

Shape of the dataset:  (59400, 14)


,rent_approval_date,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,monthly_rent,Shopping Malls,Planned MRT Stations,Existing MRT Stations,Primary Schools,location_pca,area_features_pca
0,0.67,3,10,-1.149784,-0.651406,1.344518,103.738630,1600,-1.202674,1.324998,0.300873,2.665154,1.018614,-0.585337
1,1.33,4,10,-0.101653,-1.065867,1.330186,103.938717,2250,-1.114338,0.095656,0.101009,4.392284,1.340089,1.979528
2,1.75,3,5,-1.149784,-1.646113,1.332242,103.845643,1900,2.531703,-1.183876,3.781397,1.574240,-1.128307,-1.838915
3,0.58,1,3,2.288086,0.177516,1.370239,103.962894,2850,0.597641,0.483794,-1.546040,2.435031,0.482903,-0.554034
4,1.83,3,5,-1.107859,-1.563220,1.320502,103.863341,2100,-1.073354,-2.818105,2.812144,1.728277,0.201158,0.801163


In [11]:
save_path = './data/processed/train.csv'
# move the target column to the last
df = df[[c for c in df if c not in ['monthly_rent']] + ['monthly_rent']]
df.to_csv(save_path, index=False)

save_path = './data/processed/test.csv'
df_test.to_csv(save_path, index=False)